<a href="https://colab.research.google.com/github/JeroenMulkers/mumax3-tutorial/blob/master/postprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Post processing of mumax3 output data in python

intr intro

## Getting ready

If you want to run the examples in this notebook, then mumax3 must be is installed on the system that is running this notebook. Go to [mumax.github.io/download](http://mumax.github.io/download) for more information on how to install mumax3 on your system. If you are running this notebook in a google collaboratory session, it suffices to run the cell below to install mumax3 and update the PATH environment variable. 

In [ ]:
try:
    import google.colab
except ImportError:
    pass
else:
    !wget https://mumax.ugent.be/mumax3-binaries/mumax3.10_linux_cuda10.1.tar.gz
    !tar -xvf mumax3.10_linux_cuda10.1.tar.gz
    !rm mumax3.10_linux_cuda10.1.tar.gz
    !rm -rf mumax3.10 && mv mumax3.10_linux_cuda10.1 mumax3.10
    import os
    os.environ['PATH'] += ":/content/mumax3.10"

In the examples presented in this notebook, we will use the numpy and pandas libraries for post processing mumax3 output data, and matplotlib to visualize this data. So let's import these libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## mumax3 helper functions

Let's start by writing a function which reads a mumax3 output table and puts it in a Pandas dataframe.

In [ ]:
def read_mumax3_table(filename):
    """Puts the mumax3 output table in a pandas dataframe"""

    from pandas import read_table
    
    table = read_table(filename)
    table.columns = ' '.join(table.columns).split()[1::2]
    
    return table

Mumax3 does not only write output data in a table, it can also write data to ovf fields. The function below converts all ovf files in the output directory to numpy files. These files are then loaded using the numpy.load function. 

In [ ]:
def read_mumax3_ovffiles(outputdir):
    """Load all ovffiles in outputdir into a dictionary of numpy arrays 
    with the ovffilename (without extension) as key"""
    
    from subprocess import run, PIPE, STDOUT
    from glob import glob
    from os import path
    from numpy import load

    # convert all ovf files in the output directory to numpy files
    p = run(["mumax3-convert","-numpy",outputdir+"/*.ovf"], stdout=PIPE, stderr=STDOUT)
    if p.returncode != 0:
        print(p.stdout.decode('UTF-8'))

    # read the numpy files (the converted ovf files)
    fields = {}
    for npyfile in glob(outputdir+"/*.npy"):
        key = path.splitext(path.basename(npyfile))[0]
        fields[key] = load(npyfile)
    
    return fields

The function below executes a mumax3 script and returns the data of the output table in a pandas dataframe, and the saved fields as numpy arrays.

In [ ]:
def run_mumax3(script, name, verbose=False):
    """ Executes a mumax3 script and convert ovf files to numpy files
    
    Parameters
    ----------
      script:  string containing the mumax3 input script
      name:    name of the simulation (used to name the script and output dir)
      verbose: print stdout of mumax3 when it is finished
    """
    
    from subprocess import run, PIPE, STDOUT
    from os import path

    scriptfile = name + ".txt" 
    outputdir  = name + ".out"

    # write the input script in scriptfile
    with open(scriptfile, 'w' ) as f:
        f.write(script)
    
    # call mumax3 to execute this script
    p = run(["mumax3","-f",scriptfile], stdout=PIPE, stderr=STDOUT)
    if verbose or p.returncode != 0:
        print(p.stdout.decode('UTF-8'))
        
    if path.exists(outputdir + "/table.txt"):
        table = read_mumax3_table(outputdir + "/table.txt")
    else:
        table = None
        
    fields = read_mumax3_ovffiles(outputdir)
    
    return table, fields

## Standard problem 4 revisited

Let's start by putting the mumax3 input script for standard problem 4 in the string variable `script`.

In [ ]:
script="""
SetGridsize(128, 32, 1)
SetCellsize(500e-9/128, 125e-9/32, 3e-9)

Msat  = 800e3
Aex   = 13e-12
alpha = 0.02

m = uniform(1, .1, 0)
relax()

autosave(m, 200e-12)
tableadd(e_total)
tableautosave(10e-12)

B_ext = vector(-24.6E-3, 4.3E-3, 0)
run(1e-9)
"""

Now we can execute this mumax3 script using the `run_mumax3` helper function. This function returns the output table in a pandas dataframe and a dictionary of the saved fields. If `verbose=True`, then the log output of mumax3 will be printed out when mumax3 finishes. Note that on the left side of the cell, you can check if the simulation is finished or still running.

In [ ]:
table, fields = run_mumax3( script, name="standardproblem4", verbose=False )

The table data is put in a pandas dataframe. This makes it very easy to plot and analyze the table data.

In [ ]:
print(table)

plt.figure()

nanosecond = 1e-9
plt.plot( table["t"]/nanosecond, table["mx"])
plt.plot( table["t"]/nanosecond, table["my"])
plt.plot( table["t"]/nanosecond, table["mz"])

plt.xlabel("Time (ns)")
plt.ylabel("Magnetization")

plt.show()

In [ ]:
print(sorted(fields.keys()))

def show_abs_my(m):
    my_abs = np.abs( m[1,0,:,:] )
    plt.figure()
    plt.imshow(my_abs, vmin=0, vmax=1, cmap="afmhot")
    plt.show()

show_abs_my(fields["m000001"])

## Skyrmion excitation

https://journals.aps.org/prb/pdf/10.1103/PhysRevB.90.064410

In [ ]:
diam = 50e-9
thickness = 1e-9

# NUMERICAL PARAMETERS
fmax = 100e9       # maximum frequency (in Hz) of the sinc pulse
T    = 2e-9        # simulation time (longer -> better frequency resolution)
dt   = 1/(4*fmax)  # the sample time
nx   = 64          # number of cells

# Note that this is a format string, this means that the statements inside the 
# curly brackets get evaluated by python. In this way, we insert the values of 
# the variables above in the script.
script=f"""

setgridsize({nx},{nx},1)
setcellsize({diam/nx},{diam/nx},{thickness})
setgeom(circle({diam}))

Msat = 1e6
Aex = 15e-12
Dind = 3.0e-3
Ku1 = 1e6
AnisU = vector(0,0,1)
alpha = 0.001

B_ext = vector(0, 0, 0.0005 * sinc( 2*pi*{fmax}*(t-{T/2})))
m = neelskyrmion(-1,1)
minimize()


run({T/2})
autosave(m,{dt})
run({T})
"""
 
table, fields = run_mumax3(script,"vortex")

In [ ]:

mz = np.stack([fields[key][2,0] for key in sorted(fields.keys())])

freq = np.fft.fftfreq(mz.shape[0],dt)
mzavg = np.mean(mz,axis=(1,2)) - np.mean(mz,axis=(0,1,2))
plt.semilogy(freq,np.abs(np.fft.fft(mzavg)**2))
plt.xlim(0,50e9)


In [ ]:
mz = np.stack([fields[key][2,0] for key in sorted(fields.keys())])
print(mz.shape)

freq = np.fft.fftfreq(mz.shape[0],dt)
mzfft = np.fft.fft(mz,axis=0)
mzfftsum = np.sum(np.abs(mzfft),axis=(1,2))
plt.figure()
plt.semilogy(mzfftsum)
plt.ylim(0,1.6*np.max(mzfftsum[1:]))
plt.xlim(0,100)
plt.show()

plt.figure()
plt.imshow(np.abs(mzfft[87]))
plt.show()

plt.figure()
plt.plot(np.mean(np.abs(mzfft[87][32:34]),axis=0))
plt.show()

## Ferromagnetic spinwave dispersion relation

Disregarding dipole-dipole interactions, the dispersion relation $f(k)$ in a ferromagnetic wire is given by

$$  f(k) = \frac{\gamma}{2\pi}\left[ \frac{2A}{M_s} k^2 + B \right] $$

with $A$ the exchange stiffness, $M_s$ the saturation magnetization, and $B$ an externally applied field.

In [ ]:
def freq(k,A,B,Ms):
    gamma = 1.76e11
    return (gamma/(2*np.pi)) * ( (2*A/Ms)*k**2 + B )

# plot the dispersion relation for Permalloy, without an applied field
k = np.linspace(-2e8, 2e8, 1000)
f = freq(k, A=13e-12, Ms =800e3, B=0.0)
plt.plot(k,f)
plt.xlabel("$k$ (1/m)")
plt.ylabel("$f$ (Hz)")
plt.show()

In [ ]:
# NUMERICAL PARAMETERS
fmax = 20e9        # maximum frequency (in Hz) of the sinc pulse
T    = 1e-8        # simulation time (longer -> better frequency resolution)
dt   = 1/(4*fmax)  # the sample time
dx   = 4e-9        # cellsize
nx   = 1024        # number of cells

# MATERIAL/SYSTEM PARAMETERS
Bz    = 0.2        # Bias field along the z direction
A     = 13e-12     # exchange constant
Ms    = 800e3      # saturation magnetization
alpha = 0.05       # damping parameter

# Note that this is a format string, this means that the statements inside the 
# curly brackets get evaluated by python. In this way, we insert the values of 
# the variables above in the script.
script=f"""
setgridsize({nx},1,1)
setcellsize({dx},{dx},{dx})

enabledemag = false
Aex = {A}
Msat = {Ms}
alpha = {alpha}

Bz := {Bz}
B_ext = vector(0,0,{Bz})
defregion(1,rect(2*{dx},inf))
B_ext.setregion(1, vector(0.01 * sinc( 2*pi*{fmax}*(t-{T}/2)), 0, {Bz}))

m = uniform(0,0,1) 
autosave(m,{dt})
run({T})
"""
 
table, fields = run_mumax3(script,"spinwaves")

In [ ]:
mx = np.stack([fields[key][0,0,0] for key in sorted(fields.keys())])

mx_fft = np.fft.fft2(mx)
mx_fft = np.fft.fftshift(mx_fft)

extent = [ -(2*np.pi) / (2*dx), (2*np.pi) / (2*dx), -1 / (2*dt), 1 / (2*dt)] 

gammaLL = 1.76e11
def dispersion(k):
    return A*gammaLL*k**2 /(np.pi*Ms) + gammaLL*Bz /(2*np.pi)

plt.figure()
plt.imshow(np.abs(mx_fft), extent=extent, aspect='auto', origin='lower', cmap="inferno")

#k = np.linspace(-2e8,2e8,1000)
#plt.plot(k,dispersion(k),'r--',lw=1)
#plt.axhline(gammaLL*Bz/(2*np.pi),c='g',ls='--',lw=1)

plt.xlim([-2e8,2e8])
plt.ylim([0,fmax])
plt.ylabel("$f$ (Hz)")
plt.xlabel("$k$ (1/m)")

plt.show()